# Credential Issuance and Presentation

<div class="alert alert-primary">
<b>🎯 OBJECTIVE</b><hr>
Demonstrate the process of issuing an ACDC, also known as a Verifiable Credential (VC), from an Issuer to a Holder using the Issuance and Presentation Exchange (IPEX) protocol. This involves setting up clients, defining a credential schema, creating the credential, and securely transferring it.
</div> 

## Client setup

In [ ]:
import { randomPasscode, Serder} from 'npm:signify-ts';
import { initializeSignify, 
         initializeAndConnectClient,
         createNewAID,
         addEndRoleForAID,
         generateOOBI,
         resolveOOBI,
         createTimestamp,
         DEFAULT_IDENTIFIER_ARGS,
         DEFAULT_TIMEOUT_MS,
         DEFAULT_DELAY_MS,
         DEFAULT_RETRIES,
         ROLE_AGENT,
         IPEX_GRANT_ROUTE,
         IPEX_ADMIT_ROUTE,
         IPEX_APPLY_ROUTE,
         IPEX_OFFER_ROUTE,
         SCHEMA_SERVER_HOST
       } from './scripts_ts/utils.ts';

// Clients setup
// Initialize Issuer, Holder and Verifier CLients, Create AIDs for each one, assign 'agent' role, 
// and generate OOBIs 

// Issuer Client
const issuerBran = randomPasscode()
const issuerAidAlias = 'issuerAid'
const { client: issuerClient } = await initializeAndConnectClient(issuerBran)
const { aid: issuerAid} = await createNewAID(issuerClient, issuerAidAlias, DEFAULT_IDENTIFIER_ARGS);
await addEndRoleForAID(issuerClient, issuerAidAlias, ROLE_AGENT);
const issuerOOBI = await generateOOBI(issuerClient, issuerAidAlias, ROLE_AGENT);

// Holder Client
const holderBran = randomPasscode()
const holderAidAlias = 'holderAid'
const { client: holderClient } = await initializeAndConnectClient(holderBran)
const { aid: holderAid} = await createNewAID(holderClient, holderAidAlias, DEFAULT_IDENTIFIER_ARGS);
await addEndRoleForAID(holderClient, holderAidAlias, ROLE_AGENT);
const holderOOBI = await generateOOBI(holderClient, holderAidAlias, ROLE_AGENT);

// Verifier Client
const verifierBran = randomPasscode()
const verifierAidAlias = 'verifierAid'
const { client: verifierClient } = await initializeAndConnectClient(verifierBran)
const { aid: verifierAid} = await createNewAID(verifierClient, verifierAidAlias, DEFAULT_IDENTIFIER_ARGS);
await addEndRoleForAID(verifierClient, verifierAidAlias, ROLE_AGENT);
const verifierOOBI = await generateOOBI(verifierClient, verifierAidAlias, ROLE_AGENT);

// Clients OOBI Resolution
// Resolve OOBIs to establish connections Issuer-Holder, Holder-Verifier
const issuerContactAlias = 'issuerContact';
const holderContactAlias = 'holderContact';
const verifierContactAlias = 'verifierContact';

await resolveOOBI(issuerClient, holderOOBI, holderContactAlias);
await resolveOOBI(holderClient, issuerOOBI, issuerContactAlias);
await resolveOOBI(verifierClient, holderOOBI, holderContactAlias);
await resolveOOBI(holderClient, verifierOOBI, verifierContactAlias);

// Schemas OOBI Resolution
// Resolve the Schemas from the Schema Server
const schemaContactAlias = 'schemaContact';
const schemaSaid = 'EGUPiCVO73M9worPwR3PfThAtC0AJnH5ZgwsXf6TzbVK';
const schemaOOBI = `http://vlei-server:7723/oobi/${schemaSaid}`;

await resolveOOBI(issuerClient, schemaOOBI, schemaContactAlias);
await resolveOOBI(holderClient, schemaOOBI, schemaContactAlias);
await resolveOOBI(verifierClient, schemaOOBI, schemaContactAlias);

console.log("Done.")

## Credential Issuance

### Create Credential Registry 

In [ ]:
//Create Issuer credential Registry

const issuerRegistryName = 'issuerRegistry'

const createRegistryResult = await issuerClient
    .registries()
    .create({ name: issuerAidAlias, registryName: issuerRegistryName });

const createRegistryOperation = await createRegistryResult.op();

const createRegistryResponse = await issuerClient
    .operations()
    .wait(createRegistryOperation, AbortSignal.timeout(DEFAULT_TIMEOUT_MS));

await issuerClient.operations().delete(createRegistryOperation.name);

// Listing Registries

const issuerRegistries = await issuerClient.registries().list(issuerAidAlias);
const issuerRegistry = issuerRegistries[0]
console.log(issuerRegistry)

### Retrieve and list schemas

In [ ]:
// Retrieve Schemas

const issuerSchema = await issuerClient.schemas().get(schemaSaid);

// List Schemas
// const issuerSchemas = await holderClient.schemas().list();


### Issue Credential

In [ ]:
// Issue Credential

const credentialClaims = {
    "eventName":"GLEIF Summit",
    "accessLevel":"staff",
    "validDate":"2026-10-01"
}

const issueResult = await issuerClient
    .credentials()
    .issue(
        issuerAidAlias,
        {
            ri: issuerRegistry.regk,
            s: schemaSaid,
            a: {
                i: holderAid.i,
                ...credentialClaims                
            }
        });

const issueOperation = await issueResult.op;

const issueResponse = await issuerClient
    .operations()
    .wait(issueOperation, AbortSignal.timeout(DEFAULT_TIMEOUT_MS));

await issuerClient.operations().delete(issueOperation.name);

// Credential said
const credentialSaid = issueResponse.response.ced.d

// const issuerCredentials = await issuerClient.credentials().list();
// console.log(issuerCredentials)

const issuerCredential = await issuerClient.credentials().get(credentialSaid);
console.log(issuerCredential)

### Issuer Ipex Grant

In [ ]:
// Ipex Grant

const [grant, gsigs, gend] = await issuerClient.ipex().grant({
    senderName: issuerAidAlias,
    acdc: new Serder(issuerCredential.sad), // ACDC (Credential)
    iss: new Serder(issuerCredential.iss),  // Event (Registry - ACDC)
    anc: new Serder(issuerCredential.anc),  // Event (ACDC - Issuer KEL - ISS Event) 
    ancAttachment: issuerCredential.ancatc, // Signatures
    recipient: holderAid.i,
    datetime: createTimestamp(),
});

### Issuer Grant Submit

In [ ]:

// Issuer submit grant

const submitGrantOperation = await issuerClient
    .ipex()
    .submitGrant(
        issuerAidAlias,
        grant, 
        gsigs, 
        gend,
        [holderAid.i]
    );

const submitGrantResponse = await issuerClient
    .operations()
    .wait(submitGrantOperation, AbortSignal.timeout(DEFAULT_TIMEOUT_MS));

await issuerClient.operations().delete(submitGrantOperation.name);

### Holder Credential State

In [ ]:
// Here the flow of the script transitions from the issuer to the holder
// therefore, it is required to add logic to wait for the operations to process

let credentialState;

for (let attempt = 1; attempt <= DEFAULT_RETRIES ; attempt++) {
    try{
        credentialState = await holderClient.credentials().state(issuerRegistry.regk, issuerCredential.sad.d)
        break;
    }
    catch (error){    
         console.log(`[Retry] failed on attempt #${attempt} of ${DEFAULT_RETRIES}`);
         if (attempt === DEFAULT_RETRIES) {
             console.error(`[Retry] Max retries (${DEFAULT_RETRIES}) reached.`);
             throw error; 
         }
         console.log(`[Retry] Waiting ${DEFAULT_DELAY_MS}ms before next attempt...`);
         await new Promise(resolve => setTimeout(resolve, DEFAULT_DELAY_MS));
    }
}

console.log(credentialState)

### Holder Grant notification and Exchange retrieve

In [ ]:
// Holder retrieves Grant notification

let notifications;

for (let attempt = 1; attempt <= DEFAULT_RETRIES ; attempt++) {
    try{
        notifications = await holderClient.notifications().list(
            (n) => n.a.r === IPEX_GRANT_ROUTE && n.r === false
        );
        if(notifications.notes.length === 0){ 
            throw error;
        }
        break;
        
    }
    catch (error){    
         console.log(`[Retry] failed on attempt #${attempt} of ${DEFAULT_RETRIES}`);
         if (attempt === DEFAULT_RETRIES) {
             console.error(`[Retry] Max retries (${DEFAULT_RETRIES}) reached.`);
             throw error; 
         }
         console.log(`[Retry] Waiting ${DEFAULT_DELAY_MS}ms before next attempt...`);
         await new Promise(resolve => setTimeout(resolve, DEFAULT_DELAY_MS));
    }
}

const grantNotification = notifications.notes[0]  // Only one notification expected

console.log(grantNotification)

// The grant Details can be retrieved from the exchange. The notification includes the exn said. 

const grantExchange = await holderClient.exchanges().get(grantNotification.a.d);

console.log(grantExchange)

### Holder Admits Grant

In [ ]:
// Holder admits IPEX grant

const [admit, sigs, aend] = await holderClient.ipex().admit({
    senderName: holderAidAlias,
    message: '',
    grantSaid: grantNotification.a.d!,
    recipient: issuerAid.i,
    datetime: createTimestamp(),
});

const admitOperation = await holderClient
    .ipex()
    .submitAdmit(holderAidAlias, admit, sigs, aend, [issuerAid.i]);

const admitResponse = await holderClient
    .operations()
    .wait(admitOperation, AbortSignal.timeout(DEFAULT_TIMEOUT_MS));

await holderClient.operations().delete(admitOperation.name);

// Holder Mark Grant Notification

await holderClient.notifications().mark(grantNotification.i);
console.log(await holderClient.notifications().list());

// Holder gets the Credential

await holderClient.credentials().get(issuerCredential.sad.d);

### Issuer Admit Notification

In [ ]:
// Issuer retrieves Admit notification

let notifications;

for (let attempt = 1; attempt <= DEFAULT_RETRIES ; attempt++) {
    try{
        notifications = await issuerClient.notifications().list(
            (n) => n.a.r === IPEX_ADMIT_ROUTE && n.r === false
        );
        if(notifications.notes.length === 0){ 
            throw error;
        }
        break;
    }
    catch (error){    
         console.log(`[Retry] failed on attempt #${attempt} of ${DEFAULT_RETRIES}`);
         if (attempt === DEFAULT_RETRIES) {
             console.error(`[Retry] Max retries (${DEFAULT_RETRIES}) reached.`);
             throw error; 
         }
         console.log(`[Retry] Waiting ${DEFAULT_DELAY_MS}ms before next attempt...`);
         await new Promise(resolve => setTimeout(resolve, DEFAULT_DELAY_MS));
    }
}

const admitNotification = notifications.notes[0] //Only one notification expected

// Issuer Mark Admit Notification

await issuerClient.notifications().mark(admitNotification.i);
console.log(await issuerClient.notifications().list());

### Cleanup

In [ ]:
// Issuer Remove Admit Notification
await issuerClient.notifications().delete(admitNotification.i);
console.log(await issuerClient.notifications().list());

// Holder Remove Grant Notification
await holderClient.notifications().delete(grantNotification.i);
console.log(await holderClient.notifications().list());

## Presentation

### Verifier Apply

In [ ]:
// Verifier Ipex Apply (Presentation request)

const [apply, sigs, _] = await verifierClient.ipex().apply({
    senderName: verifierAidAlias,
    schemaSaid: schemaSaid,
    attributes: { eventName:'GLEIF Summit' },
    recipient: holderAid.i,
    datetime: createTimestamp(),
});

const applyOperation = await verifierClient
    .ipex()
    .submitApply(verifierAidAlias, apply, sigs, [holderAid.i]);

const applyResponse = await verifierClient
    .operations()
    .wait(applyOperation, AbortSignal.timeout(DEFAULT_TIMEOUT_MS));

await verifierClient.operations().delete(applyOperation.name);

### Holder Apply Notification and Exchange

In [ ]:
// holder IPEX apply receive and offer

let notifications;

for (let attempt = 1; attempt <= DEFAULT_RETRIES ; attempt++) {
    try{
        notifications = await holderClient.notifications().list(
            (n) => n.a.r === IPEX_APPLY_ROUTE && n.r === false
        );
        if(notifications.notes.length === 0){ 
            throw error;
        }
        break;
    }
    catch (error){    
         console.log(`[Retry] failed on attempt #${attempt} of ${DEFAULT_RETRIES}`);
         if (attempt === DEFAULT_RETRIES) {
             console.error(`[Retry] Max retries (${DEFAULT_RETRIES}) reached.`);
             throw error; 
         }
         console.log(`[Retry] Waiting ${DEFAULT_DELAY_MS}ms before next attempt...`);
         await new Promise(resolve => setTimeout(resolve, DEFAULT_DELAY_MS));
    }
}

const applyNotification = notifications.notes[0] // Only one notification expected

console.log(applyNotification)

const applyExchange = await holderClient.exchanges().get(applyNotification.a.d);

console.log(applyExchange)

const applyExchangeSaid = applyExchange.exn.d;

// Holder Mark Apply Notification
await holderClient.notifications().mark(applyNotification.i);
console.log(await holderClient.notifications().list());



### Holder Find Matching Credential

In [ ]:
// The apply operation from the verifier asks for a specific credential 
// (matching schema and attribute values) as presented below:
//
//   schemaSaid: schemaSaid,
//   attributes: { eventName:'GLEIF Summit' },
// 
// This code snippet creates a credential filter accordingly. Taking the
// criteria from the applyExchange message received by the holder

let filter: { [x: string]: any } = { '-s': applyExchange.exn.a.s };
for (const key in applyExchange.exn.a.a) {
    filter[`-a-${key}`] = applyExchange.exn.a.a[key];
}

console.log(filter)

const matchingCredentials = await holderClient.credentials().list({ filter });

console.log(matchingCredentials)

### Holder offer

In [ ]:
const [offer, sigs, end] = await holderClient.ipex().offer({
    senderName: holderAidAlias,
    recipient: verifierAid.i,
    acdc: new Serder(matchingCredentials[0].sad),
    applySaid: applyExchangeSaid,
    datetime: createTimestamp(),
});

const offerOperation = await holderClient
    .ipex()
    .submitOffer(holderAidAlias, offer, sigs, end, [
        verifierAid.i,
    ]);

const offerResponse = await holderClient
    .operations()
    .wait(offerOperation, AbortSignal.timeout(DEFAULT_TIMEOUT_MS));

await holderClient.operations().delete(offerOperation.name);



### Verifier - Handle Offer Notification and Agree 

In [ ]:
// verifier receive offer and agree

let notifications;

for (let attempt = 1; attempt <= DEFAULT_RETRIES ; attempt++) {
    try{
        notifications = await verifierClient.notifications().list(
            (n) => n.a.r === IPEX_OFFER_ROUTE && n.r === false
        );
        if(notifications.notes.length === 0){ 
            throw error;
        }
        break;
    }
    catch (error){    
         console.log(`[Retry] failed on attempt #${attempt} of ${DEFAULT_RETRIES}`);
         if (attempt === DEFAULT_RETRIES) {
             console.error(`[Retry] Max retries (${DEFAULT_RETRIES}) reached.`);
             throw error; 
         }
         console.log(`[Retry] Waiting ${DEFAULT_DELAY_MS}ms before next attempt...`);
         await new Promise(resolve => setTimeout(resolve, DEFAULT_DELAY_MS));
    }
}

const offerNotification = notifications.notes[0];

const offerExchange = await verifierClient.exchanges().get(offerNotification.a.d);

let offerExchangeSaid = offerExchange.exn.d;

// Holder Mark Apply Notification
await verifierClient.notifications().mark(offerNotification.i);

console.log(await verifierClient.notifications().list());

In [ ]:
const [agree, sigs, _] = await verifierClient.ipex().agree({
    senderName: verifierAidAlias,
    recipient: holderAid.i,
    offerSaid: offerExchangeSaid,
    datetime: createTimestamp(),
});

const agreeOperation = await verifierClient
    .ipex()
    .submitAgree(verifierAidAlias, agree, sigs, [holderAid.i]);

const agreeResponse = await verifierClient
    .operations()
    .wait(agreeOperation, AbortSignal.timeout(DEFAULT_TIMEOUT_MS));

await verifierClient.operations().delete(agreeOperation.name);